In [ ]:
from funkcije import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
combined_data = load_data()

In [ ]:
combined_data['DatumPN'] = pd.to_datetime(combined_data['DatumPN'], format='%d.%m.%Y')
combined_data['Weekday'] = combined_data['DatumPN'].dt.day_name()
combined_data['Day'] = combined_data['DatumPN'].dt.day
combined_data['Month'] = combined_data['DatumPN'].dt.month
combined_data['Year'] = combined_data['DatumPN'].dt.year

In [ ]:
classification_colors = {
    'Z MATERIALNO ŠKODO': '#377eb8',  
    'Z LAŽJO TELESNO POŠKODBO': '#4daf4a', 
    'S HUDO TELESNO POŠKODBO': '#ff7f00',  
    'S SMRTNIM IZIDOM': '#e41a1c'  
}

In [ ]:
first_date = combined_data['DatumPN'].min()
last_date = combined_data['DatumPN'].max()

days = (last_date - first_date).days
weeks = days // 7
months = (last_date.year - first_date.year) * 12 + (last_date.month - first_date.month)
years = combined_data['Year'].nunique()
days, weeks, months, years 

# Najnevarnejši meseci

In [ ]:
unique_accidents = combined_data.groupby('ZaporednaStevilkaPN').first()

stacked_data = unique_accidents.groupby(['Month', 'KlasifikacijaNesrece']).size().unstack(fill_value=0)

ordered_columns = ['Z MATERIALNO ŠKODO', 'Z LAŽJO TELESNO POŠKODBO', 'S HUDO TELESNO POŠKODBO', 'S SMRTNIM IZIDOM']
stacked_data[ordered_columns] = stacked_data[ordered_columns] / years

stacked_data = stacked_data[ordered_columns]
stacked_data = stacked_data.reindex(
    range(1, 13),
    axis=0
)

stacked_data.plot(
    kind='barh',
    stacked=True,
    color=[classification_colors.get(col, 'gray') for col in stacked_data.columns],
    figsize=(10, 5)
)

plt.title('Nesreče povprečno po mesecih', fontsize=14)
plt.xlabel('Število nesreč', fontsize=12)
plt.ylabel('Mesec', fontsize=12)
plt.yticks(range(0, 12), ['Jan', 'Feb', 'Mar', 'Apr', 'Maj', 'Jun', 'Jul', 'Avg', 'Sep', 'Okt', 'Nov', 'Dec'])
plt.legend(title='Klasifikacija nesreč', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.tight_layout()
plt.show()

# Najnevarnejši dnevi v tednu

In [ ]:
unique_accidents = combined_data.groupby('ZaporednaStevilkaPN').first()

stacked_data = unique_accidents.groupby(['Weekday', 'KlasifikacijaNesrece']).size().unstack(fill_value=0)

ordered_columns = ['Z MATERIALNO ŠKODO', 'Z LAŽJO TELESNO POŠKODBO', 'S HUDO TELESNO POŠKODBO', 'S SMRTNIM IZIDOM']
stacked_data[ordered_columns] = stacked_data[ordered_columns] / weeks

stacked_data = stacked_data[ordered_columns]
stacked_data = stacked_data.reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    axis=0
)

stacked_data.plot(
    kind='barh',
    stacked=True,
    color=[classification_colors.get(col, 'gray') for col in stacked_data.columns],
    figsize=(10, 5)
)

plt.title('Nesreče povprečno na dan v tednu', fontsize=14)
plt.xlabel('Število nesreč', fontsize=12)
plt.ylabel('Dan v tednu', fontsize=12)
plt.yticks(range(0, 7), ['Pon', 'Tor', 'Sre', 'Čet', 'Pet', 'Sob', 'Ned'])
plt.legend(title='Klasifikacija nesreč', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.tight_layout()
plt.show()

# Vpliv praznikov

In [ ]:
def getEasterDate(year):
    a = year % 19
    b = year // 100
    c = year % 100
    d = b // 4
    e = b % 4
    f = (b + 8) // 25
    g = (b - f + 1) // 3
    h = (19 * a + b - d - g + 15) % 30
    i = c // 4
    k = c % 4
    l = (32 + 2 * e + 2 * i - h - k) % 7
    m = (a + 11 * h + 22 * l) // 451
    month = (h + l - 7 * m + 114) // 31
    day = ((h + l - 7 * m + 114) % 31) + 1
    return month, day

In [ ]:
# https://www.gov.si/teme/drzavni-prazniki-in-dela-prosti-dnevi/
holidays = dict() # {'datum': 'praznik'}
def add_holiday(date, name):
    if date < first_date or date > last_date:
        return
    if date in holidays:
        if isinstance(holidays[date], list):
            holidays[date].append(name)
        else:
            holidays[date] = [holidays[date], name]
    else:
        holidays[date] = name

for year in range(first_date.year, last_date.year + 1):
    add_holiday(pd.to_datetime(f"{year}-01-01"), 'novo leto')
    add_holiday(pd.to_datetime(f"{year}-01-02"), 'novo leto')
    add_holiday(pd.to_datetime(f"{year}-02-08"), 'Prešernov dan, slovenski kulturni praznik')
    add_holiday(pd.to_datetime(f"{year}-04-27"), 'dan upora proti okupatorju')
    add_holiday(pd.to_datetime(f"{year}-05-01"), 'praznik dela')
    add_holiday(pd.to_datetime(f"{year}-05-02"), 'praznik dela')
    add_holiday(pd.to_datetime(f"{year}-06-08"), 'dan Primoža Trubarja')
    add_holiday(pd.to_datetime(f"{year}-06-25"), 'dan državnosti')
    add_holiday(pd.to_datetime(f"{year}-08-15"), 'Marijino vnebovzetje')
    add_holiday(pd.to_datetime(f"{year}-08-17"), 'združitev prekmurskih Slovencev z matičnim narodom')
    add_holiday(pd.to_datetime(f"{year}-09-15"), 'priključitev Primorske k matični domovini')
    add_holiday(pd.to_datetime(f"{year}-09-23"), 'dan slovenskega športa')
    add_holiday(pd.to_datetime(f"{year}-10-25"), 'dan suverenosti')
    add_holiday(pd.to_datetime(f"{year}-10-31"), 'dan reformacije')
    add_holiday(pd.to_datetime(f"{year}-11-01"), 'dan spomina na mrtve')
    add_holiday(pd.to_datetime(f"{year}-11-10"), 'dan znanosti')
    add_holiday(pd.to_datetime(f"{year}-11-23"), 'dan Rudolfa Maistra')
    add_holiday(pd.to_datetime(f"{year}-12-25"), 'božič')
    add_holiday(pd.to_datetime(f"{year}-12-26"), 'dan samostojnosti in enotnosti')

    month, day = getEasterDate(year)
    easter_date = pd.to_datetime(f"{year}-{month}-{day}")

    add_holiday(easter_date, 'velikonočna nedelja')
    add_holiday(easter_date + pd.Timedelta(days=1), 'velikonočni ponedeljek')
    add_holiday(easter_date + pd.Timedelta(days=49), 'binkoštna nedelja')

In [ ]:
def get_holiday_occurence_count(holiday):
    if pd.isna(holiday):
        return days - len(holidays)
    i = 0
    for date in holidays:
        if isinstance(holidays[date], list):
            if holiday in holidays[date]:
                i += 1
        else:
            if holidays[date] == holiday:
                i += 1
    return i

In [ ]:
holiday_accidents = combined_data.copy()
holiday_accidents['Holiday'] = holiday_accidents['DatumPN'].map(holidays)
holiday_accidents = holiday_accidents.explode('Holiday') # 2014-06-08 je dan Rudolfa Maistra in binkoštna nedelja
holiday_accidents = holiday_accidents.groupby('ZaporednaStevilkaPN').first()

stacked_data = holiday_accidents.groupby(['Holiday', 'KlasifikacijaNesrece'], dropna=False).size().unstack(fill_value=0)

ordered_columns = ['Z MATERIALNO ŠKODO', 'Z LAŽJO TELESNO POŠKODBO', 'S HUDO TELESNO POŠKODBO', 'S SMRTNIM IZIDOM']
for col in ordered_columns:
    stacked_data[col] = stacked_data.apply(
        lambda row: row[col] / get_holiday_occurence_count(row.name),
        axis=1
    )

stacked_data = stacked_data.reindex(
    [
        'novo leto', 
        'Prešernov dan, slovenski kulturni praznik', 
        'velikonočna nedelja',
        'velikonočni ponedeljek',
        'dan upora proti okupatorju', 
        'praznik dela', 
        'binkoštna nedelja',
        'dan Primoža Trubarja', 
        'dan državnosti', 
        'Marijino vnebovzetje',
        'združitev prekmurskih Slovencev z matičnim narodom',
        'priključitev Primorske k matični domovini',
        'dan slovenskega športa',
        'dan suverenosti',
        'dan reformacije',
        'dan spomina na mrtve',
        'dan znanosti',
        'dan Rudolfa Maistra',
        'božič',
        'dan samostojnosti in enotnosti',
        np.nan
    ],
    axis=0
)

stacked_data.plot(
    kind='barh',
    stacked=True,
    color=[classification_colors.get(col, 'gray') for col in stacked_data.columns],
    figsize=(10, 5)
)

plt.title('Nesreče povprečno na praznik na leto', fontsize=14)
plt.xlabel('Število nesreč', fontsize=12)
plt.ylabel('Praznik', fontsize=12)
plt.yticks(range(0, 21), [
        'novo leto', 
        'Prešernov dan, slovenski kulturni praznik', 
        'velikonočna nedelja',
        'velikonočni ponedeljek',
        'dan upora proti okupatorju', 
        'praznik dela', 
        'binkoštna nedelja',
        'dan Primoža Trubarja', 
        'dan državnosti', 
        'Marijino vnebovzetje',
        'združitev prekmurskih Slovencev z matičnim narodom',
        'priključitev Primorske k matični domovini',
        'dan slovenskega športa',
        'dan suverenosti',
        'dan reformacije',
        'dan spomina na mrtve',
        'dan znanosti',
        'dan Rudolfa Maistra',
        'božič',
        'dan samostojnosti in enotnosti',
        'ni praznik'
    ])
plt.legend(title='Klasifikacija nesreč', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.tight_layout()
plt.show()